In [72]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
# !pip install -q pyomo
# !apt-get install -y -qq glpk-utils
# import pyomo.environ as pyo
# !wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
# !unzip -o -q couenne-linux64

import pyomo.environ as pyo


In [109]:
class StarNetwork:
    def __init__(self, n_devices, n_types):
        self.n_types = n_types
        self.n_devices = n_devices
        self.groups = [set() for i in range(n_types)]
        for i in range(n_devices):
            self.groups[np.random.choice(n_types)].add(i)

        self.fast_link = set(np.random.choice(N, N//2, False))
        self.slow_link = set(range(N)) - self.fast_link


        self.delay = np.random.uniform(5, 10, N)
        self.delay[list(self.slow_link)] = self.delay[list(self.slow_link)] + np.random.uniform(10, 20, len(self.slow_link))

        self.bw = np.random.uniform(100, 200, N)     
        self.bw[list(self.slow_link)] = np.random.uniform(20, 50, len(self.slow_link))

        self.L = np.zeros((N, N))
        self.R = np.zeros((N, N))


        for i in range(N):
            self.L[i, i] = 0
            self.R[i, i] = 0
            for j in range(i+1, N):
                self.L[i,j] = self.delay[i] + self.delay[j]
                self.R[i,j] = 1/self.bw[i] + 1/self.bw[j]
                self.L[j, i] = self.L[i, j]
                self.R[j, i] = self.R[i, j]
                
        self.C1 = 0.2
        self.C2 = 30


    def central_delay(self, n_bytes, i, j):
        if i == j:
            return 0
        return n_bytes * self.C1 + self.C2

    def communication_delay(self, n_bytes, i, j):
        return self.L[i,j] + n_bytes * self.R[i,j] + self.central_delay(n_bytes, i, j)
    
    def get_delay_matrix(self):
        B = self.L + self.C2
        for i in range(N):
            B[i,i] = 0
        return B
    
    def get_rate_matrix(self, nodes=None):
        R =  self.R + self.C1
        for i in range(N):
            R[i,i] = 0
        if nodes:
            return R[nodes][:,nodes]
        return R


class Program:
    def __init__(self, n_operators, network):
        self.n_operators = n_operators
        self.P = nx.DiGraph()
        self.P.add_edges_from([(0, 1), (0, 2), (0, 3), (1, 4), (1, 6), (2, 4), (2, 3), (3, 5), (4, 5), (5, 6)])
#         self.P.add_edges_from([(i, i+1) for i in range(n_operators-1)])

        self.B = np.zeros((n_operators, n_operators))
        for e in self.P.edges:
            self.P.edges[e]['bytes'] = np.random.uniform(500, 1000)
            self.B[e] = self.P.edges[e]['bytes']

        k = network.n_types
        self.placement_constraints = [np.random.choice(k, k//2 + (np.random.sample()> 0.5) * 1 - (np.random.sample()> 0.5) * 1) for i in range(n_operators)]
        for i in range(n_operators):
            self.placement_constraints[i] = set().union(*[network.groups[j] for j in self.placement_constraints[i]])

        self.T = np.zeros([n_operators, network.n_devices])
        for i in range(n_operators):
            self.T[i, list(self.placement_constraints[i])] = np.random.exponential(50, len(self.placement_constraints[i]))
            self.T[self.T==0] = np.inf

        self.pinned = [np.random.choice(list(self.placement_constraints[0])), np.random.choice(list(self.placement_constraints[-1]))]



def get_mapped_node(map, i):
    return np.where(map[i] == 1)[0][0]


def evaluate(mapping, program, network):
    for o in nx.topological_sort(program.P):
        if program.P.in_degree(o) == 0:
            program.P.nodes[o]['t'] = program.T[o, get_mapped_node(mapping, o)]
            program.P.nodes[o]['critical'] = None
        else:
            max_time = 0
            critical_node = None
            des = get_mapped_node(mapping, o)
            for in_edge in program.P.in_edges(o):
                bytes = program.B[in_edge]
                s = get_mapped_node(mapping, in_edge[0])
                c = network.communication_delay(bytes, s, des)
                dl = c + program.P.nodes[in_edge[0]]['t']
                if dl > max_time:
                    max_time = dl
                    critical_node = in_edge[0]
            program.P.nodes[o]['t'] = max_time + program.T[o, des]
            program.P.nodes[o]['critical'] = critical_node
    o = list(nx.topological_sort(program.P))[-1]
    latency = program.P.nodes[o]['t'] 
    critical_path = [o]

    n = program.P.nodes[o]['critical']
    while n is not None:
        critical_path.append(n)
        n = program.P.nodes[n]['critical']
    critical_path.reverse()
    return latency, critical_path

def evaluate_maxP(mapping, program, network):
    for o in program.P.nodes:
        des = get_mapped_node(mapping, o)
        program.P.nodes[o]['c'] = program.T[o, des]
            
    for e in program.P.edges:
        d1 = get_mapped_node(mapping, e[0])
        d2 = get_mapped_node(mapping, e[1])
        b = program.B[e]
        program.P.edges[e]['c'] = network.communication_delay(b, d1, d2)
    
    for o in program.P.nodes:
        if program.P.in_degree(o) == 0:
            for e in program.P.out_edges(o):
                program.P.edges[e]['c'] += program.P.nodes[o]['c']
            continue
        if program.P.out_degree(o) == 0:
            for e in program.P.in_edges(o):
                program.P.edges[e]['c'] += program.P.nodes[o]['c']
            continue
        for e in program.P.in_edges(o):
            program.P.edges[e]['c'] += program.P.nodes[o]['c']/2
        for e in program.P.out_edges(o):
            program.P.edges[e]['c'] += program.P.nodes[o]['c']/2
    
    critical_path = nx.dag_longest_path(program.P, 'c')
    latency = 0
    for i in range(len(critical_path)-1):
        latency += program.P.edges[critical_path[i], critical_path[i+1]]['c']
    
    return latency, critical_path

def qlp_linear(pinned, placement_constraints, B, T, D, R):
    M = T.shape[0]
    N = T.shape[1]    
    model = pyo.ConcreteModel(name='Linear constraint')
    model.x = pyo.Var(range(M), range(N), domain=pyo.Binary)


    def _obj(m):
        summ = 0
        for i in range(M):
            for j in range(N):
                if T[i,j] < np.inf:
                    summ += T[i,j] * m.x[i,j]
        for i in range(M):
            for j in range(M):
                if B[i,j] > 0:
                    for k in range(N):
                        for l in range(N):
                            summ += m.x[i,k] * m.x[j,l] *(B[i,j] *  R[l, k] +  D[l, k])
        return summ

    model.obj = pyo.Objective(rule = _obj, sense=pyo.minimize)

    model.c = pyo.ConstraintList()
    model.c.add(model.x[0, pinned[0]] == 1)
    model.c.add(model.x[M-1, pinned[1]] == 1)
    for i in range(1,M-1):
        model.c.add(sum(model.x[i,j] for j in range(N)) == 1)
        model.c.add(sum(model.x[i,j] for j in list(placement_constraints[i])) == 1)

# model.pprint()

    baron = pyo.SolverFactory('baron', executable='~/baron-osx64/baron')
    result_obj = baron.solve(model, tee=False)
    solution = [j for i in range(M) for j in range(N) if pyo.value(model.x[i,j]) > 0]
    print(solution)
    return solution



In [110]:
k = 5
N = 50
M = 7


network = StarNetwork(N, k)
program = Program(M, network)

mapping = np.zeros((M, N))
mapping[0, program.pinned[0]] = 1
mapping[-1, program.pinned[-1]] = 1

print(program.pinned)




[29, 19]


In [117]:
def iterative(mapping, program, network):
    map = np.copy(mapping)
    constraints = program.placement_constraints
    to_be_mapped = [i for i in range(program.n_operators) if np.sum(mapping[i]) < 1]

    for o in to_be_mapped:
        map[o, np.random.choice(list(constraints[o]))] = 1
  
    last_latency, critical_path = evaluate(map, program, network)
    
    count = 100
    while True:
        order = list(range(1, len(critical_path)-1))
        np.random.shuffle(order)
        for i in order:
            o = critical_path[i]
            s1 = critical_path[i-1]
            s2 = critical_path[i+1]
            d = d1 = get_mapped_node(map, o)
            d1 = get_mapped_node(map, s1)
            d2 = get_mapped_node(map, s2)
            c_p_c =  network.communication_delay(program.B[s1, o], d1, d) + network.communication_delay(program.B[o, s2], d, d2) + program.T[o, d]
            choices = list(constraints[o]) 
            for n in choices:
                new_cpc = network.communication_delay(program.B[s1, o], d1, n) + network.communication_delay(program.B[o, s2], n, d2) + program.T[o, n]
                if new_cpc < c_p_c:
                    c_p_c = new_cpc
                    d = n
            map[o] = 0
            map[o, d] = 1
        cur_latency, critical_path = evaluate(map, program, network)
        if cur_latency < last_latency:
            last_latency = cur_latency
        elif count:
            count -= 1
        else:
            break
    return map, last_latency

mapp, Lat = iterative(mapping, program, network)
print([get_mapped_node(mapp, i) for i in range(M)])
print(evaluate(mapp, program, network))
# count = 0
# avg = 0
# for i in range(200):
#     mapp, lat = iterative(mapping, program, network)
#     avg += lat
#     if lat == Lat:
#         count += 1
# print(f"Probability of finding the optimal = {count/200}")
# print(f"Solution average latency = {avg/200}")

[29, 14, 29, 21, 21, 21, 19]
(969.3066197569501, [0, 1, 4, 5, 6])


In [119]:
def iterative_qlp(mapping, program, network):
    mapp = np.copy(mapping)
    constraints = program.placement_constraints
    to_be_mapped = [i for i in range(program.n_operators) if np.sum(mapping[i]) < 1]

    for o in to_be_mapped:
        mapp[o, np.random.choice(list(constraints[o]))] = 1
  
    last_latency, critical_path = evaluate(mapp, program, network)
    best_mapp = np.copy(mapp)
    
    count = 10
    while True:
        p_constraints = [constraints[i] for i in critical_path]
        B = program.B[critical_path][:, critical_path]
        T = program.T[critical_path]
        D = network.get_delay_matrix()
        R = network.get_rate_matrix()
        sol = qlp_linear(program.pinned, p_constraints,B, T, D, R)
        for i in range(1, len(critical_path)-1):
            mapp[critical_path[i]] = 0
            mapp[critical_path[i], sol[i]] = 1
        cur_latency, cur_path = evaluate(mapp, program, network)
        if set(critical_path) == set(cur_path):
            break
        if not count:
            break
        count -=  1

        critical_path = cur_path
        if cur_latency < last_latency:
            last_latency = cur_latency
            best_mapp = np.copy(mapp)
        
    return best_mapp, last_latency

mapp, Lat = iterative_qlp(mapping, program, network)
print([get_mapped_node(mapp, i) for i in range(M)])
print(evaluate(mapp, program, network))

[29, 14, 35, 35, 19]
[29, 29, 29, 15, 19]
[29, 14, 35, 35, 19]
[29, 14, 29, 29, 35, 15, 19]
(1181.2643615319541, [0, 1, 4, 5, 6])


In [116]:
def exhaustive(mapping, program, network):
    to_be_mapped = []
    constraints = program.placement_constraints
    for i in nx.topological_sort(program.P):
        if np.sum(mapping[i]) < 1:
            to_be_mapped.append(i)
    l = len(to_be_mapped)
  
    def helper(to_be, idx, constraints):
        if idx == len(to_be) - 1:
            for node in constraints[to_be[idx]]:
                yield [node]
        else:
            for node in constraints[to_be[idx]]:
                partial_mapping = helper(to_be, idx+1, constraints)
                for p_m in partial_mapping:
                    p_m.append(node)
                    yield p_m 
  
    min_L = np.inf
    min_mapping = None
    mapp = np.copy(mapping)
  
    for mapped in helper(to_be_mapped, 0, constraints):
        for i in range(l):
            mapp[to_be_mapped[i]] = 0
            mapp[to_be_mapped[i], mapped[-1-i]] = 1
        latency_i, _ = evaluate(mapp, program, network)
        if latency_i < min_L:
            min_L = latency_i
            min_mapping = np.copy(mapp)
    return min_mapping, min_L


mapp, Lat = exhaustive(mapping, program, network)
print([get_mapped_node(mapp, i) for i in range(M)])
print(evaluate(mapp, program, network))


[29, 14, 5, 35, 35, 35, 19]
(945.6213757364159, [0, 1, 4, 5, 6])
